Привет! меня зовут Люман Аблаев. Сегодня я проверю твой проект.
<br> Дальнейшее общение будет происходить на "ты" если это не вызывает никаких проблем.
<br> Желательно реагировать на каждый мой комментарий ('исправил', 'не понятно как исправить ошибку', ...)
<br> Пожалуйста, не удаляй комментарии ревьюера, так как они повышают качество повторного ревью.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:


<div class="alert alert-block alert-success">
<b>Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Если требуются исправления. Работа не может быть принята с красными комментариями.
</div>

-------------------

Будет очень хорошо, если ты будешь помечать свои действия следующим образом:
<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> ..
</div>
<div class="alert alert-block alert-warning">
<b>Изменения:</b> Были внесены следующие изменения ...
</div>

<font color='orange' style='font-size:24px; font-weight:bold'>Общее впечатление</font>
* Спасибо за  качественную работу, было интересно ее проверять.
- Я оставил некоторые советы надеюсь они будут полезными или интересными.
- Есть небольшие недочеты, но думаю у тебя не составить труда их исправить
- Жду обновленную работу!

# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## 1. Загрузка и подготовка данных

In [127]:
# Подключаем все необходимые библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Разбиение на обучающую, валидационную и тестовую выборку
from sklearn.model_selection import train_test_split
# Применим кроссвалидацию для повышения качеств обучения
from sklearn.model_selection import cross_val_score

# Масштабируемость модели
from sklearn.preprocessing import StandardScaler

# и для машинного обучения разными способами (по условию мы выбираем линейную регрессию):
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error
from sklearn.metrics import (
    # Точность модели
    accuracy_score,
    # Матрицы ошибок (для борьбы с дисбалансом)
    confusion_matrix, 
    # Полнота
    recall_score, 
    # Точность
    precision_score, 
    # F1-мера
    f1_score,
    # Метрика AUC-ROC
    roc_auc_score,
    roc_curve,
    # MSE
    mean_squared_error
)

# Контроль выборки
from sklearn.utils import shuffle

<div class="alert alert-block alert-success">
<b>Успех:</b> Импорты  на месте
</div>

In [128]:
# Получаем датасеты
try:
    # Локальный файл
    try:
        df_0 = pd.read_csv('geo_data_0.csv')
        df_1 = pd.read_csv('geo_data_1.csv')
        df_2 = pd.read_csv('geo_data_2.csv')
    # Файл с яндекса
    except:
        df_0 = pd.read_csv('/datasets/geo_data_0.csv')
        df_1 = pd.read_csv('/datasets/geo_data_1.csv')
        df_2 = pd.read_csv('/datasets/geo_data_2.csv')

    # Узнаем, что в них
    df_0.info()
    df_1.info()
    df_2.info()
except:
    print('Отсутствует датасет. Проверьте путь файла')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

In [129]:
df_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [130]:
df_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [131]:
df_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [132]:
# Проверим на наличие пропущенных значений
df_0.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [133]:
df_1.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [134]:
df_2.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

Поскольку мы будем применять метод линейной регрессии, значит рассмотрим корреляцию

In [135]:
pip install pandas_profiling

Note: you may need to restart the kernel to use updated packages.


In [136]:
# Будем это применять, т.к. pandas_profiling уже устарело, но работает аналоично как это 
import ydata_profiling

In [ ]:
# Рассмотрим корреляцию 3-ех датасетов
ydata_profiling.ProfileReport(df_0)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
ydata_profiling.ProfileReport(df_1)

In [ ]:
ydata_profiling.ProfileReport(df_2)

Все в порядке, теперь можно приступать к обучению и проверку модели

<div class="alert alert-block alert-success">
<b>Успех:</b> Данные осмотрены, но можно было бы еще как минимум изучить корреляции, так как мы планируем использовать линейные модели, а для них это важно. 
</div>


<div class="alert alert-block alert-info">
<b>Совет:</b>  Советую обратить внимание на библиотеки sweetviz и pandas_profiling помогут в проведении более тщательного EDA анализа. Исследовательский анализ можно делать и с помощью ручного вызова функций дефолтных библиотек. Данные библиотеки хороши для максимизации комфорта презентации результатов анализа бизнес-пользователям.  

Очень просты в использоовании, на вход кладется датафрейм: pandas_profiling.ProfileReport(df) 
</div>

<div class="alert alert-block alert-warning">
<b>V2 Изменения:</b> Добавлена библиотека pandas_profiling для изучения корреляции трех датасетов
</div>


<div class="alert alert-block alert-success">
    
<b>Успех[2]:</b> Главное во время обучения не злоупотреблять, чтобы руки не забыл как делать EDA анализ)
</div>

<div class="alert alert-block alert-warning">
<b>V3 Изменения:</b> Ок, ок, без проблем, постараюсь)
</div>

## 2. Обучение и проверка модели

In [ ]:
# Убираем лишний признак
df_0 = df_0[['f0', 'f1', 'f2', 'product']]
df_1 = df_1[['f0', 'f1', 'f2', 'product']]
df_2 = df_2[['f0', 'f1', 'f2', 'product']]

In [ ]:
# Объединим в один датасет в виде списка для удобства автоматизации
full_df = [df_0, df_1, df_2]

In [ ]:
# Эта переменная будет сохранять все RMSE модели для всех регионов 
best_RMSE_for_all_model_averange_for_all_region = []

# А эта будет сохранять все RMSE модели
best_RMSE_for_all_model_averange = []

In [ ]:
# Эта - все предсказания модели для всех регионов
best_predict_for_all_model_averange_for_all_region = []

# Аналогично, все предсказания модели
best_predict_for_all_model_averange = []

In [ ]:
features = []
target = []

for df in full_df:
    features.append(df.drop('product', axis=1))
    target.append(df['product'])

### 2.1. Разобьем данные на обучающую и валидационную выборки в соотношении 75:25

In [ ]:
# Как обычно, будем разделять 75% данных для обучающей выборки и 25% валидационной выборки

features_train = []
features_valid = []
target_train = []
target_valid = []

for i in range(0, 3):
    f_t, f_v, t_t, t_v = train_test_split(features[i], target[i], test_size=(1 - 0.75), random_state=12345)
    features_train.append(f_t)
    features_valid.append(f_v)
    target_train.append(t_t)
    target_valid.append(t_v)

### 2.2. Обучим модель и сделаем предсказания на валидационной выборке, 
### 2.3. Также сохраним предсказания и правильные ответы на валидационной выборки

Попробуем выявлять предсказания и RMSE методом LinearRegression

In [ ]:
%%time
# Применяем метод линейной регрессии
model = LinearRegression()

for i in range(0, 3):
    # Обучим модель на тренировочной выборке
    model.fit(features_train[i], target_train[i]) 

    # Получим предсказания модели на валидационной выборке
    predictions_valid = model.predict(features_valid[i]) 

    # Посчитаем значение метрики RMSE на валидационной выборке
    result = mean_squared_error(target_valid[i], predictions_valid) ** 0.5
    
    print('-'*79)
    print('Регион №', i + 1)
    print('RMSE модели линейной регрессии на валидационной выборке:', result)
    print('Предсказание модели:', predictions_valid.mean())
    
    # Сохраним все значения RMSE
    best_RMSE_for_all_model_averange.append(result)
    # И предсказания
    best_predict_for_all_model_averange.append(predictions_valid)
    
print('-'*79)

У перовго региона RMSE небольшое, но ощутимое. Значит где-то 37 объём запасов в скважине (тыс. баррелей) присутствует отклонение

А у второго - самое минимальное отклонение, можно рассмотреть этот регион как лучший

У третьего - больше отклонений первого региона

Здесь получается, что можно рассмотреть второй регион как минимальное отклонение

А предсказания у второго региона самое низкие остальных

<div class="alert alert-block alert-success">
<b>Успех:</b> Шаг проделан лаконично, все метрики посчитаны корректно
</div> 


In [ ]:
# Сохраним лучшие RMSE всех регионов
best_RMSE_for_all_model_averange_for_all_region.append(best_RMSE_for_all_model_averange[:])
# И предсказания всех регионов
best_predict_for_all_model_averange_for_all_region.append(best_predict_for_all_model_averange[:])

In [ ]:
# Очистим, чтобы заполняли новые значения других регионов
best_RMSE_for_all_model_averange.clear()

best_predict_for_all_model_averange.clear()

In [ ]:
print(best_predict_for_all_model_averange_for_all_region)

In [ ]:
print(best_RMSE_for_all_model_averange_for_all_region)

<div class="alert alert-block alert-danger">
<b>Ошибка: </b> в ТЗ четко указано использовать линейную регрессию, использлвание другие моделей здесь избыточно
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка: </b>  Тоже самое
</div>

<div class="alert alert-block alert-warning">
<b>V2 Изменения:</b> Убраны 2 модели, оставили только модель линейной регрессии
</div>


<div class="alert alert-block alert-success">
    
<b>Успех[2]:</b> 👍
</div>



### 2.4. Напечатаем на экране средний запас предсказанного сырья и RMSE модели

Попробуем составить общую картинку всех регионов и рассмотреть

In [ ]:
print('='*79, end='\n\n')
print('LinearRegression:')
for j in range(0, 3):
    print('Регион №', j + 1)
    print('Лучшие RMSE:', best_RMSE_for_all_model_averange_for_all_region[0][j])
    print('Лучшие средние предсказания:', best_predict_for_all_model_averange_for_all_region[0][j].mean())
print()
print('='*79, end='\n\n')

### 2.5. Проанализируем результаты и сделаем вывод
Что и требовалось доказать, у второго региона получилось самое маленькое отклонение остальных регионов

## 3. Подготовка к расчёту прибыли

### Условия задачи

- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

### 3.1. Все ключевые значения для расчётов сохраним в отдельных переменных

In [ ]:
# С помощью машинного обучения выбирали 200 лучшие точки 
BEST_POINT_FOR_PRODUCT = 200
# Бюджет 10 млрд рублей
BUDGET = 10_000_000_000
# Доход 450 тыс рублей с каждой единицы продукты (объем указывается в тыс баррелей)
INCOME = 450000
# Вероятность убытки 2.5%
PROBABILITY_OF_LOSSES = 0.025

<div class="alert alert-block alert-info">
    
<b>Совет:</b> Константы принято записывать в ВЕРХНЕМ РЕГИСТРЕ =) Читем здесь https://pythonworld.ru/osnovy/pep-8-rukovodstvo-po-napisaniyu-koda-na-python.html#section-29
    
А большие значение можно записывать так: `10_000_000_000`.
</div> 

<div class="alert alert-block alert-warning">
<b>V2 Изменения:</b> Переменные были переименованы в верхнем регистре
</div>


<div class="alert alert-block alert-success">
    
<b>Успех[2]:</b> 👍
</div>



### 3.2. Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе.

In [ ]:
# Здесь будут храняться средний запас сырья каждого региона
average_stock_of_raw_products = []

for i in range(0, 3):
    average_stock_of_raw_products.append(full_df[i]['product'].mean())
    print('Ср. запас сырья в ', i + 1, 'регионе: ', round(average_stock_of_raw_products[i], 4), 'млн. баррелей')

In [ ]:
# Также выявляем выделенные деньги на постройку одной скважины
budget_one_per_well = BUDGET / BEST_POINT_FOR_PRODUCT
print('Выделенные деньги на постройку одной скважины:', budget_one_per_well, 'рублей')

In [ ]:
# Теперь рассчитаем достаточный объем сырья
sufficient_volume_of_raw_products = BEST_POINT_FOR_PRODUCT / INCOME
print('Объем каждой скважины (Для безубыточный разработки):', round(sufficient_volume_of_raw_products, 4), 'тыс. баррелей')

In [ ]:
# Теперь сравним полученный объем сырья со средним запасом в каждом регионе
# Но в ходе отладки вяснился, что данных очень много и пришлось запихнуть в самом датасете
# Сохраняем полученный объем в каждом регионе в каждом датасете

for i in range(0, 3):
    print('='*79)
    print('Регион №', i + 1)
    full_df[i]['resulting_volume_of_raw_materials'] = full_df[i]['product'] * INCOME
    print(full_df[i]['resulting_volume_of_raw_materials'])
print('='*79)

In [ ]:
# Сохранили и теперь находим возврат на инвестиции, т.к. нужно узнать, когда же инвестиционные вложения окупятся
# И эти ROI заполняем в текущий датасет
# Формула ROI = (итоговая прибыль - инвестиции) / инвестици =
# = (resulting_volume_of_raw_materials - budget_one_per_well) / budget_one_per_well

for i in range(0, 3):
    print('='*79)
    print('Регион №', i + 1)
    full_df[i]['return_on_investments'] = (full_df[i]['resulting_volume_of_raw_materials'] - budget_one_per_well) / budget_one_per_well
    # Будет удобнее выразить через процент
    print(round(full_df[i]['return_on_investments'] * 100, 4))
print('='*79)

In [ ]:
# Теперь узнаем, в каком регионе больше всего попадаются отрицательные значения ROI

for i in range(0, 3):
    count_negative_product = 0
    print('='*79)
    print('Регион №', i + 1)

    for ROI in full_df[i]['return_on_investments']:
        # Если в этой точке попадется отрицательные вложения, тогда в копилку попадается
        if(ROI < 0):
            count_negative_product += 1

    # Сохраняем кол-во точек с отрицательными значениями
    print('Кол-во точек с отрицательными значениями:', count_negative_product)
    print('Средние ROI:', round(full_df[i]['return_on_investments'].mean() * 100, 4), end='%\n')

print('='*79)


### 3.3. Напишем выводы по этапу подготовки расчёта прибыли

Здесь получается, что инвенстировать второй регион не оказалось выгодным условием, т.к. большая часть является убыточной. Лучше всего оказалось третий регион

<div class="alert alert-block alert-success">
<b>Успех:</b> Значение для безубыточной разработки посчитано верно, с выводом согласен  
</div>

## 4. Расчёт прибыли и рисков. 

## Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели

###  4.1. Выберем скважины с максимальными значениями предсказаний

In [ ]:
# Выберем скважину третьего региона, т.к. он лучше остальных по предсказанию
best_well = best_predict_for_all_model_averange_for_all_region[0][2]
print('Лучшие предсказания скважины третьего региона:\n', best_well, '\nИ средние значение предсказания:', round(best_well.mean(), 4))

### 4.2. Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям

In [ ]:
target_value_of_volume_of_raw_materials = best_well.sum()
print('Объем сырья:', round(target_value_of_volume_of_raw_materials, 4), 'тыс. баррелей')

### 4.3. Рассчитаем прибыль для полученного объёма сырья

In [ ]:
# Прибыль для полученного объема = (целевое значение объема * доход с одного барреля) - бюджет на разработку скважин в регионе
profit_for_received_volume_of_raw_materials = (target_value_of_volume_of_raw_materials * INCOME) - BUDGET
print('Прибыль:', profit_for_received_volume_of_raw_materials, 'рублей')
print('Или в другом виде:', round(profit_for_received_volume_of_raw_materials / 1_000_000, 2), 'млн. рублей')

In [ ]:
# Теперь создадим функцию, которая выполняет примерно аналогично
def profit_for_received(target, probabilities, count):
    # Отсортируем вероятность для удобства
    probs_sorted = pd.Series(probabilities, index=target.index).sort_values(ascending=False)
    # Чтобы выбрать ответы, возьмем их индексы
    selected = target[probs_sorted.index][:count].sum()

    profit_for_received_volume_of_raw_materials = (selected * INCOME) - BUDGET

    print('Объем сырья:', round(selected, 4), 'тыс. баррелей')
     
    print('Прибыль:', profit_for_received_volume_of_raw_materials, 'рублей')
    print('Или в другом виде:', round(profit_for_received_volume_of_raw_materials / 1_000_000, 2), 'млн. рублей')


<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Для выбора 200 лучших скважин нужно использовать предсказанные значения. А вот для подсчета прибыли должны использоваться настоящие значения соответствующих ячеек.

</div>



<div class="alert alert-block alert-success">
    
<b>Успех[2]:</b> Есть контакт
</div>



In [ ]:
# Поскольку по условию с помощью машинного обучения выбираем 200 лучших для разработки

for i in range(0, 3):
    print('='*79)
    print('Регион №', i + 1)
    profit_for_received(target_valid[i], best_predict_for_all_model_averange_for_all_region[0][i], 200)
print('='*79)

<div class="alert alert-block alert-warning">
<b>V2 Изменения:</b> Исправлена функция
</div>

Здесь видно, что первый регион оказался прибыльным по сравнению остальных. Самый низкий показатель по прибыли оказался второй регион

## 5. Посчитайте риски и прибыль для каждого региона

### 5.1. Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли

 ### 5.2. И найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль

In [ ]:
# Теперь дублируем выше функцию, которая выполняет примерно аналогично, 
# но будет возвращать значения, т.к. предыдующая функия ничего не возвращает

def profit_for_bootstrap(target, probabilities, count):
    # Отсортируем вероятность для удобства
    probs_sorted = pd.Series(probabilities, index=target.index).sort_values(ascending=False)
    # Чтобы выбрать ответы, возьмем их индексы
    selected = target[probs_sorted.index][:count].sum()

    profit_for_received_volume_of_raw_materials = (selected * INCOME) - BUDGET

    return profit_for_received_volume_of_raw_materials

In [ ]:
# Квантиль убытки
lower = []
# Квантиль прибыли
upper = []
negative_count_region = []
values_region = []

for j in range(0, 3):
    # Сохраним значения 99%-квантилей в переменной values
    values = []
    
    state = np.random.RandomState(12345)

    for i in range(1000):
        # Чтобы bootstrap работал, нужно работать только с Series, поэтому преобразуем
        best_predict_series = pd.Series(best_predict_for_all_model_averange_for_all_region[0][j], index=target_valid[j].index)
        subsample = best_predict_series.sample(n=500, replace=True, random_state=state)
        values.append(profit_for_bootstrap(subsample, best_predict_series, 200))

    
    lower.append(pd.Series(values).quantile(PROBABILITY_OF_LOSSES))
    upper.append(pd.Series(values).quantile(1 - PROBABILITY_OF_LOSSES))


    # Расччитаем убытки
    negative_count_region.append((pd.Series(values) < 0).mean() * 100)
    
    # Сохраняем это значение для средней выручки региона
    values_region.append(values)


<div class="alert alert-block alert-danger">
<b>Ошибка[2]:</b> Тебе в функцию прибыли нужно передавать 500 предсказаний, и 500 соответствующих таргетов, а ты вторым аргументом передаешь не совсем то, что нужно)

</div>


<div class="alert alert-block alert-info">
<b>Совет[2]: </b>
    
Риски можно лаконично считать в 1 строку: `(pd.Series(values) < 0).mean() * 100`
</div>                                                              



<div class="alert alert-block alert-warning">
<b>V3 Изменения:</b> А да, не заметил. Оптимизировал фрагмент кода, который считывает риск
</div>

In [ ]:
for i in range(0, 3):
    print('='*79)
    print('Регион №', i + 1)
    print('Вероятность 95% прибыль в этом регионе:', round(lower[i] / 1_000_000, 2), ' - ', round(upper[i] / 1_000_000, 2), 'млн. рублей')
    print("Средняя выручка млн. рублей:", round(pd.Series(values_region[i]).mean() / 1_000_000, 2))
    print('Вероятность убытки в этом регионе:', round(negative_count_region[i] / len(values), 4))
print('='*79)

### 5.3. Выводы: предложим регион для разработки скважин и обоснуем выбор

Выше можно посмотреть, что первый регион имеет низкий процент убытки по сравнению остальных, а средняя выручка занимает второе место остальных. Если хотим большую выручку и не боимся потерпевать убытки, тогда рассмотрим второй регион

<div class="alert alert-block alert-danger">
<b>Ошибка: </b>  Здесь нужно реализовать технику бутстрап:
    
1) У тебя 1000 итераций - на каждой итерации нужно сэмплировать 500 скважин, то есть поправить `frac` нужно
    
2) 500 значений таргетов и 500 предсказаний нужно будет передать в функцию прибыль (что бы по предсказаниям прошла сортировка, а прибыль посчиталась по таргетам)
    
3) Значение прибыли добавляем в список values (никакой квантиль здесь добавть не нужно)
    
4) После цикла считаем доверительный интервал - это 0.025 и 0.975 квантили, а не 0.05 и 0.95
    
5) Считаем  риск убытков - долю отрицательной прибыли, то есть какова доля отрицательных значений содержится в списке values
</div>

<div class="alert alert-block alert-warning">

<b> V1 Изменения:</b> Здесь я тут немного не понял, отсюда можно вычерпнуть с темы 3: "Запуск новой функциональности", урок "Бутстреп для анализа A/B-теста"?
    
</div>

<div class="alert alert-block alert-warning">
<b> V2 Изменения:</b> Подправил bootstrap и добавлен вывод
</div>


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован